# 3D Lattice Graph Connectivity Analysis

This notebook analyzes the connectivity properties of the 3D lattice graph for UAS (drone) path planning over Manhattan NYC. The graph represents a multi-layer airspace with nodes at different altitudes and horizontal connections forming a lattice structure.

## Objectives:
- Analyze graph connectivity patterns
- Identify connected components
- Evaluate path efficiency and reachability
- Calculate network centrality measures
- Visualize connectivity structure

## 1. Import Required Libraries

In [ ]:
# Import necessary libraries for graph analysis and visualization
import pickle
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set up plotting parameters
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("Libraries imported successfully!")
print(f"NetworkX version: {nx.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Analysis started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 2. Load Graph from Pickle File

In [ ]:
# Load the graph from the pickle file
pickle_filename = "regular_lattice_graph.pkl"

try:
    with open(pickle_filename, 'rb') as f:
        graph_data = pickle.load(f)
    
    print("✅ Pickle file loaded successfully!")
    print(f"Data type: {type(graph_data)}")
    
    # Extract the graph and metadata
    if isinstance(graph_data, dict):
        G = graph_data.get('graph')
        metadata = graph_data.get('metadata', {})
        print(f"\n📊 Metadata found:")
        for key, value in metadata.items():
            print(f"   {key}: {value}")
    else:
        G = graph_data
        metadata = {}
        print("   No metadata found - using graph directly")
    
    print(f"\n🎯 Graph loaded: {type(G)}")
    print(f"   NetworkX Graph: {isinstance(G, nx.Graph)}")
    
except FileNotFoundError:
    print(f"❌ Error: {pickle_filename} not found in current directory")
    print("Please ensure the pickle file is in the same folder as this notebook")
except Exception as e:
    print(f"❌ Error loading pickle file: {e}")

## 3. Basic Graph Information

In [ ]:
# Basic graph information and structure analysis
print("🔍 BASIC GRAPH INFORMATION")
print("=" * 50)

# Graph type and basic stats
print(f"Graph Type: {type(G).__name__}")
print(f"Is Directed: {G.is_directed()}")
print(f"Is Multigraph: {G.is_multigraph()}")
print(f"Number of Nodes: {G.number_of_nodes():,}")
print(f"Number of Edges: {G.number_of_edges():,}")

if G.number_of_nodes() > 0:
    print(f"Average Degree: {sum(dict(G.degree()).values()) / G.number_of_nodes():.2f}")

# Node attributes analysis
print(f"\n📋 NODE ATTRIBUTES:")
if G.nodes():
    sample_node = list(G.nodes())[0]
    node_attrs = G.nodes[sample_node]
    print(f"Sample node: {sample_node}")
    print(f"Node attributes: {list(node_attrs.keys())}")
    
    # Analyze node attributes
    for attr in node_attrs.keys():
        values = [G.nodes[n].get(attr) for n in G.nodes()]
        unique_values = set([v for v in values if v is not None])
        print(f"  - {attr}: {len(unique_values)} unique values")

# Edge attributes analysis
print(f"\n🔗 EDGE ATTRIBUTES:")
if G.edges():
    sample_edge = list(G.edges())[0]
    edge_attrs = G.edges[sample_edge]
    print(f"Sample edge: {sample_edge}")
    print(f"Edge attributes: {list(edge_attrs.keys())}")
    
    # Analyze edge attributes
    for attr in edge_attrs.keys():
        values = [G.edges[e].get(attr) for e in G.edges()]
        unique_values = set([v for v in values if v is not None])
        print(f"  - {attr}: {len(unique_values)} unique values")

# Degree distribution
degrees = [d for n, d in G.degree()]
print(f"\n📊 DEGREE DISTRIBUTION:")
print(f"Min degree: {min(degrees)}")
print(f"Max degree: {max(degrees)}")
print(f"Mean degree: {np.mean(degrees):.2f}")
print(f"Median degree: {np.median(degrees):.2f}")
print(f"Standard deviation: {np.std(degrees):.2f}")

# Count nodes by degree
degree_counts = Counter(degrees)
print(f"\nDegree frequency:")
for degree in sorted(degree_counts.keys()):
    count = degree_counts[degree]
    percentage = (count / len(degrees)) * 100
    print(f"  Degree {degree}: {count} nodes ({percentage:.1f}%)")

### 3.1 Edge Direction Verification

In [ ]:
# Detailed verification of directed vs undirected edges
print("🔍 DIRECTED vs UNDIRECTED EDGE VERIFICATION")
print("=" * 50)

# Check if the graph data contains both directed and undirected information
total_edges = G.number_of_edges()
directed_edges = 0
undirected_edges = 0
bidirectional_pairs = 0

print(f"Graph is classified as: {'Directed' if G.is_directed() else 'Undirected'}")
print(f"Total edges in graph: {total_edges:,}")

if G.is_directed():
    # For directed graphs, check for bidirectional connections
    print(f"\n🔄 ANALYZING DIRECTED GRAPH EDGE PATTERNS:")
    
    edge_pairs = {}  # Track (u,v) and (v,u) pairs
    
    for u, v in G.edges():
        # Create a canonical pair representation
        pair = tuple(sorted([u, v]))
        
        if pair not in edge_pairs:
            edge_pairs[pair] = []
        edge_pairs[pair].append((u, v))
    
    # Analyze the patterns
    unidirectional_pairs = 0
    bidirectional_pairs = 0
    
    for pair, edges in edge_pairs.items():
        if len(edges) == 1:
            unidirectional_pairs += 1
        elif len(edges) == 2:
            bidirectional_pairs += 1
        else:
            print(f"⚠️  Unusual: {len(edges)} edges for pair {pair}")
    
    print(f"  Unidirectional pairs: {unidirectional_pairs:,} (single direction)")
    print(f"  Bidirectional pairs: {bidirectional_pairs:,} (both directions)")
    print(f"  Total node pairs with edges: {len(edge_pairs):,}")
    
    # Calculate edge counts
    directed_edges = unidirectional_pairs + (bidirectional_pairs * 2)
    
    print(f"\n📊 EDGE COUNT BREAKDOWN:")
    print(f"  Unidirectional edges: {unidirectional_pairs:,}")
    print(f"  Bidirectional edges: {bidirectional_pairs * 2:,} (from {bidirectional_pairs:,} pairs)")
    print(f"  Total directed edges: {directed_edges:,}")
    
    # Verify totals
    if directed_edges == total_edges:
        print(f"  ✅ Edge count verification: PASSED")
    else:
        print(f"  ❌ Edge count verification: FAILED (expected {total_edges}, calculated {directed_edges})")
    
    # Calculate what this would look like as undirected
    equivalent_undirected_edges = unidirectional_pairs + bidirectional_pairs
    print(f"\n🔄 IF CONVERTED TO UNDIRECTED:")
    print(f"  Equivalent undirected edges: {equivalent_undirected_edges:,}")
    print(f"  Reduction factor: {total_edges / equivalent_undirected_edges:.2f}x")
    
    # Check for self-loops
    self_loops = list(nx.selfloop_edges(G))
    print(f"\n🔄 SELF-LOOPS:")
    print(f"  Number of self-loops: {len(self_loops)}")
    if len(self_loops) > 0 and len(self_loops) <= 10:
        print(f"  Self-loop nodes: {[edge[0] for edge in self_loops]}")
    elif len(self_loops) > 10:
        print(f"  First 10 self-loop nodes: {[edge[0] for edge in self_loops[:10]]}")

else:
    # For undirected graphs, all edges are undirected
    print(f"\n↔️  ANALYZING UNDIRECTED GRAPH:")
    undirected_edges = total_edges
    
    print(f"  All edges are undirected: {undirected_edges:,}")
    
    # Check what this would look like as directed
    print(f"\n🔄 IF CONVERTED TO DIRECTED:")
    print(f"  Would create: {undirected_edges * 2:,} directed edges")
    print(f"  Expansion factor: 2.0x")
    
    # Check for self-loops
    self_loops = list(nx.selfloop_edges(G))
    print(f"\n🔄 SELF-LOOPS:")
    print(f"  Number of self-loops: {len(self_loops)}")
    if len(self_loops) > 0 and len(self_loops) <= 10:
        print(f"  Self-loop nodes: {[edge[0] for edge in self_loops]}")
    elif len(self_loops) > 10:
        print(f"  First 10 self-loop nodes: {[edge[0] for edge in self_loops[:10]]}")

# Summary statistics
print(f"\n📋 SUMMARY:")
print(f"  Graph type: {'Directed' if G.is_directed() else 'Undirected'}")
print(f"  Total edges: {total_edges:,}")

if G.is_directed():
    print(f"  Bidirectional pairs: {bidirectional_pairs:,}")
    print(f"  Unidirectional pairs: {unidirectional_pairs:,}")
    connectivity_ratio = bidirectional_pairs / (bidirectional_pairs + unidirectional_pairs) if (bidirectional_pairs + unidirectional_pairs) > 0 else 0
    print(f"  Bidirectionality ratio: {connectivity_ratio:.1%}")
else:
    print(f"  All edges are inherently bidirectional")

print(f"\n✅ Edge direction verification completed!")

### 3.2 Vertical Layer Connectivity Analysis

In [ ]:
# Analyze vertical connections between altitude layers in the 3D lattice
print("🏗️ VERTICAL LAYER CONNECTIVITY ANALYSIS")
print("=" * 50)

# Extract layer information from node attributes
layer_nodes = {}  # layer -> list of nodes
node_layers = {}  # node -> layer
layer_coordinates = {}  # layer -> set of (x,y) coordinates

print("📊 ANALYZING NODE LAYER DISTRIBUTION:")

# Analyze node attributes to identify layers
if G.nodes():
    sample_node = list(G.nodes())[0]
    node_attrs = G.nodes[sample_node]
    
    # Check for common layer attribute names
    layer_attr = None
    for attr in ['layer', 'altitude', 'z', 'level', 'floor']:
        if attr in node_attrs:
            layer_attr = attr
            break
    
    if layer_attr:
        print(f"Found layer attribute: '{layer_attr}'")
        
        # Group nodes by layer
        for node in G.nodes():
            layer = G.nodes[node].get(layer_attr, 'unknown')
            
            if layer not in layer_nodes:
                layer_nodes[layer] = []
            layer_nodes[layer].append(node)
            node_layers[node] = layer
            
            # Also collect x,y coordinates if available
            if layer not in layer_coordinates:
                layer_coordinates[layer] = set()
            
            x_coord = G.nodes[node].get('x', G.nodes[node].get('lon', None))
            y_coord = G.nodes[node].get('y', G.nodes[node].get('lat', None))
            
            if x_coord is not None and y_coord is not None:
                layer_coordinates[layer].add((x_coord, y_coord))
        
        print(f"Identified {len(layer_nodes)} distinct layers:")
        for layer in sorted(layer_nodes.keys()):
            nodes_in_layer = len(layer_nodes[layer])
            coords_in_layer = len(layer_coordinates.get(layer, set()))
            print(f"  Layer {layer}: {nodes_in_layer} nodes, {coords_in_layer} unique coordinates")
    
    else:
        print("⚠️ No standard layer attribute found. Attempting coordinate-based analysis...")
        
        # Try to infer layers from z-coordinate or altitude
        z_values = set()
        for node in G.nodes():
            z_val = None
            for attr in ['z', 'altitude', 'alt', 'height', 'elevation']:
                if attr in G.nodes[node]:
                    z_val = G.nodes[node][attr]
                    break
            
            if z_val is not None:
                z_values.add(z_val)
                node_layers[node] = z_val
                
                if z_val not in layer_nodes:
                    layer_nodes[z_val] = []
                layer_nodes[z_val].append(node)
        
        if z_values:
            print(f"Inferred {len(z_values)} layers from z-coordinates: {sorted(z_values)}")
        else:
            print("❌ Could not identify layer structure from node attributes")

# Analyze vertical connections
if len(layer_nodes) > 1:
    print(f"\n🔗 VERTICAL CONNECTION ANALYSIS:")
    
    vertical_edges = []
    horizontal_edges = []
    same_layer_edges = []
    
    # Classify edges by their vertical nature
    for u, v in G.edges():
        u_layer = node_layers.get(u, 'unknown')
        v_layer = node_layers.get(v, 'unknown')
        
        if u_layer != 'unknown' and v_layer != 'unknown':
            if u_layer == v_layer:
                same_layer_edges.append((u, v))
            else:
                vertical_edges.append((u, v, u_layer, v_layer))
        else:
            horizontal_edges.append((u, v))  # Unknown layer classification
    
    print(f"Total edges analyzed: {len(list(G.edges()))}")
    print(f"Same-layer (horizontal) edges: {len(same_layer_edges)}")
    print(f"Vertical (inter-layer) edges: {len(vertical_edges)}")
    print(f"Unclassified edges: {len(horizontal_edges)}")
    
    # Analyze vertical edge patterns
    if vertical_edges:
        print(f"\n📈 VERTICAL EDGE PATTERNS:")
        
        layer_transitions = {}  # (from_layer, to_layer) -> count
        vertical_node_pairs = {}  # (node1, node2) -> edge_info
        
        for u, v, u_layer, v_layer in vertical_edges:
            # Create canonical layer transition
            transition = tuple(sorted([u_layer, v_layer]))
            if transition not in layer_transitions:
                layer_transitions[transition] = 0
            layer_transitions[transition] += 1
            
            # Track node pairs for bidirectionality analysis
            node_pair = tuple(sorted([u, v]))
            if node_pair not in vertical_node_pairs:
                vertical_node_pairs[node_pair] = []
            vertical_node_pairs[node_pair].append((u, v, u_layer, v_layer))
        
        # Display transition patterns
        print(f"Layer transition patterns:")
        for transition, count in sorted(layer_transitions.items()):
            layer1, layer2 = transition
            print(f"  Layers {layer1} ↔ {layer2}: {count} edges")
        
        # Check bidirectionality of vertical connections
        bidirectional_vertical = 0
        unidirectional_vertical = 0
        
        for node_pair, edges in vertical_node_pairs.items():
            if len(edges) == 1:
                unidirectional_vertical += 1
            elif len(edges) == 2:
                bidirectional_vertical += 1
            else:
                print(f"⚠️ Unusual vertical connection: {len(edges)} edges for {node_pair}")
        
        print(f"\n🔄 VERTICAL BIDIRECTIONALITY:")
        print(f"  Bidirectional vertical pairs: {bidirectional_vertical}")
        print(f"  Unidirectional vertical pairs: {unidirectional_vertical}")
        
        total_vertical_pairs = bidirectional_vertical + unidirectional_vertical
        if total_vertical_pairs > 0:
            vertical_bidir_ratio = bidirectional_vertical / total_vertical_pairs
            print(f"  Vertical bidirectionality ratio: {vertical_bidir_ratio:.1%}")
        
        # Check if vertical edges are undirected (bidirectional)
        if G.is_directed():
            undirected_vertical_connections = bidirectional_vertical
            directed_vertical_connections = unidirectional_vertical
            
            print(f"\n🎯 VERTICAL EDGE NATURE:")
            print(f"  Effectively undirected vertical connections: {undirected_vertical_connections}")
            print(f"  Truly directed vertical connections: {directed_vertical_connections}")
            
            if undirected_vertical_connections > 0:
                print(f"  ✅ Found {undirected_vertical_connections} undirected vertical connections!")
            else:
                print(f"  ❌ No undirected vertical connections found")
        else:
            print(f"\n🎯 VERTICAL EDGE NATURE:")
            print(f"  All vertical edges are inherently undirected: {len(vertical_edges)}")
            print(f"  ✅ All vertical connections are undirected by graph nature!")
        
        # Analyze vertical connectivity by coordinate
        if layer_coordinates and len(layer_coordinates) > 1:
            print(f"\n📍 COORDINATE-BASED VERTICAL ANALYSIS:")
            
            # Check if same (x,y) coordinates exist across layers
            common_coordinates = None
            for layer in layer_coordinates:
                if common_coordinates is None:
                    common_coordinates = layer_coordinates[layer].copy()
                else:
                    common_coordinates &= layer_coordinates[layer]
            
            if common_coordinates:
                print(f"  Common (x,y) coordinates across layers: {len(common_coordinates)}")
                print(f"  This suggests a regular 3D lattice structure")
                
                # Sample a few vertical stacks
                sample_coords = list(common_coordinates)[:5]
                for coord in sample_coords:
                    vertical_stack = []
                    for layer in sorted(layer_nodes.keys()):
                        for node in layer_nodes[layer]:
                            x_coord = G.nodes[node].get('x', G.nodes[node].get('lon', None))
                            y_coord = G.nodes[node].get('y', G.nodes[node].get('lat', None))
                            if x_coord is not None and y_coord is not None:
                                if (x_coord, y_coord) == coord:
                                    vertical_stack.append((node, layer))
                    
                    if len(vertical_stack) > 1:
                        print(f"    Vertical stack at {coord}: {len(vertical_stack)} nodes")
                        
                        # Check connections within this stack
                        stack_connections = 0
                        for i in range(len(vertical_stack) - 1):
                            node1, layer1 = vertical_stack[i]
                            for j in range(i + 1, len(vertical_stack)):
                                node2, layer2 = vertical_stack[j]
                                if G.has_edge(node1, node2) or G.has_edge(node2, node1):
                                    stack_connections += 1
                        
                        max_possible = len(vertical_stack) * (len(vertical_stack) - 1) // 2
                        if not G.is_directed():
                            max_possible = len(vertical_stack) - 1  # Adjacent layers only typically
                        
                        print(f"      Connections in stack: {stack_connections}")
            else:
                print(f"  No common coordinates across layers - irregular structure")
    
    else:
        print(f"\n❌ No vertical edges found between layers")
        print(f"   All {len(same_layer_edges)} edges are within the same layer")

else:
    print(f"\n⚠️ Cannot analyze vertical connections:")
    if len(layer_nodes) <= 1:
        print(f"   Only {len(layer_nodes)} layer(s) identified")
    else:
        print(f"   Layer structure could not be determined")

print(f"\n✅ Vertical layer connectivity analysis completed!")

## 4. Connectivity Analysis

In [ ]:
# Comprehensive connectivity analysis
print("🌐 GRAPH CONNECTIVITY ANALYSIS")
print("=" * 50)

# Basic connectivity checks
if G.is_directed():
    print("🔄 DIRECTED GRAPH CONNECTIVITY:")
    is_strongly_connected = nx.is_strongly_connected(G)
    is_weakly_connected = nx.is_weakly_connected(G)
    print(f"Strongly Connected: {is_strongly_connected}")
    print(f"Weakly Connected: {is_weakly_connected}")
    
    if not is_strongly_connected:
        num_strongly_connected = nx.number_strongly_connected_components(G)
        print(f"Number of Strongly Connected Components: {num_strongly_connected}")
    
    if not is_weakly_connected:
        num_weakly_connected = nx.number_weakly_connected_components(G)
        print(f"Number of Weakly Connected Components: {num_weakly_connected}")
        
else:
    print("↔️ UNDIRECTED GRAPH CONNECTIVITY:")
    is_connected = nx.is_connected(G)
    print(f"Connected: {is_connected}")
    
    if not is_connected:
        num_components = nx.number_connected_components(G)
        print(f"Number of Connected Components: {num_components}")

# Node and Edge connectivity (for smaller graphs)
if G.number_of_nodes() < 1000:
    try:
        if G.is_directed():
            node_connectivity = nx.node_connectivity(G)
            edge_connectivity = nx.edge_connectivity(G)
        else:
            node_connectivity = nx.node_connectivity(G)
            edge_connectivity = nx.edge_connectivity(G)
        
        print(f"\n🔗 CONNECTIVITY MEASURES:")
        print(f"Node Connectivity: {node_connectivity}")
        print(f"Edge Connectivity: {edge_connectivity}")
        
    except nx.NetworkXError as e:
        print(f"\n⚠️ Connectivity measures not available: {e}")
else:
    print(f"\n⚠️ Graph too large ({G.number_of_nodes()} nodes) for detailed connectivity measures")

# Density analysis
if G.is_directed():
    density = nx.density(G)
    max_edges = G.number_of_nodes() * (G.number_of_nodes() - 1)
else:
    density = nx.density(G)
    max_edges = G.number_of_nodes() * (G.number_of_nodes() - 1) // 2

print(f"\n📊 DENSITY ANALYSIS:")
print(f"Graph Density: {density:.6f}")
print(f"Current Edges: {G.number_of_edges():,}")
print(f"Maximum Possible Edges: {max_edges:,}")
print(f"Edge Utilization: {(G.number_of_edges() / max_edges * 100):.2f}%")

# Isolated nodes
isolated_nodes = list(nx.isolates(G))
print(f"\n🏝️ ISOLATED NODES:")
print(f"Number of Isolated Nodes: {len(isolated_nodes)}")
if len(isolated_nodes) > 0 and len(isolated_nodes) <= 10:
    print(f"Isolated Nodes: {isolated_nodes}")
elif len(isolated_nodes) > 10:
    print(f"First 10 Isolated Nodes: {isolated_nodes[:10]}")

# Self-loops and multiple edges
if hasattr(G, 'number_of_selfloops'):
    self_loops = nx.number_of_selfloops(G)
    print(f"\n🔄 SELF-LOOPS:")
    print(f"Number of Self-loops: {self_loops}")

print(f"\n✅ Connectivity analysis completed!")

## 5. Connected Components Analysis

In [ ]:
# Detailed analysis of connected components
print("🧩 CONNECTED COMPONENTS ANALYSIS")
print("=" * 50)

if G.is_directed():
    # Strongly connected components
    print("💪 STRONGLY CONNECTED COMPONENTS:")
    scc = list(nx.strongly_connected_components(G))
    scc_sizes = [len(component) for component in scc]
    
    print(f"Number of Strongly Connected Components: {len(scc)}")
    print(f"Largest SCC size: {max(scc_sizes) if scc_sizes else 0}")
    print(f"Smallest SCC size: {min(scc_sizes) if scc_sizes else 0}")
    print(f"Average SCC size: {np.mean(scc_sizes):.2f}" if scc_sizes else "N/A")
    
    # Size distribution
    scc_size_counts = Counter(scc_sizes)
    print(f"\nSCC Size Distribution:")
    for size in sorted(scc_size_counts.keys(), reverse=True)[:10]:
        count = scc_size_counts[size]
        percentage = (count / len(scc)) * 100
        print(f"  Size {size}: {count} components ({percentage:.1f}%)")
    
    # Weakly connected components
    print(f"\n🤝 WEAKLY CONNECTED COMPONENTS:")
    wcc = list(nx.weakly_connected_components(G))
    wcc_sizes = [len(component) for component in wcc]
    
    print(f"Number of Weakly Connected Components: {len(wcc)}")
    print(f"Largest WCC size: {max(wcc_sizes) if wcc_sizes else 0}")
    print(f"Smallest WCC size: {min(wcc_sizes) if wcc_sizes else 0}")
    print(f"Average WCC size: {np.mean(wcc_sizes):.2f}" if wcc_sizes else "N/A")

else:
    # Connected components for undirected graph
    print("🔗 CONNECTED COMPONENTS:")
    cc = list(nx.connected_components(G))
    cc_sizes = [len(component) for component in cc]
    
    print(f"Number of Connected Components: {len(cc)}")
    if cc_sizes:
        print(f"Largest Component size: {max(cc_sizes)} nodes ({max(cc_sizes)/G.number_of_nodes()*100:.1f}% of total)")
        print(f"Smallest Component size: {min(cc_sizes)} nodes")
        print(f"Average Component size: {np.mean(cc_sizes):.2f}")
        
        # Size distribution
        cc_size_counts = Counter(cc_sizes)
        print(f"\nComponent Size Distribution:")
        for size in sorted(cc_size_counts.keys(), reverse=True)[:10]:
            count = cc_size_counts[size]
            percentage = (count / len(cc)) * 100
            nodes_in_size = size * count
            nodes_percentage = (nodes_in_size / G.number_of_nodes()) * 100
            print(f"  Size {size}: {count} components ({percentage:.1f}% of components, {nodes_percentage:.1f}% of nodes)")

# Analyze largest component in detail
if G.is_directed():
    largest_wcc = max(wcc, key=len) if wcc else set()
    largest_component_size = len(largest_wcc)
    component_type = "Weakly Connected Component"
else:
    largest_cc = max(cc, key=len) if cc else set()
    largest_component_size = len(largest_cc)
    component_type = "Connected Component"

print(f"\n🎯 LARGEST {component_type.upper()} ANALYSIS:")
print(f"Size: {largest_component_size} nodes ({largest_component_size/G.number_of_nodes()*100:.1f}% of total)")

if largest_component_size > 0:
    if G.is_directed():
        largest_subgraph = G.subgraph(largest_wcc)
    else:
        largest_subgraph = G.subgraph(largest_cc)
    
    print(f"Edges in largest component: {largest_subgraph.number_of_edges()}")
    print(f"Density of largest component: {nx.density(largest_subgraph):.6f}")
    
    # Degree statistics for largest component
    largest_degrees = [d for n, d in largest_subgraph.degree()]
    if largest_degrees:
        print(f"Average degree in largest component: {np.mean(largest_degrees):.2f}")
        print(f"Degree range in largest component: {min(largest_degrees)} - {max(largest_degrees)}")

print(f"\n✅ Connected components analysis completed!")

## 6. Path Analysis

In [ ]:
# Shortest path and reachability analysis
print("🛤️ PATH ANALYSIS")
print("=" * 50)

# For large graphs, we'll sample nodes for path analysis
max_nodes_for_path_analysis = 500
nodes_list = list(G.nodes())

if len(nodes_list) <= max_nodes_for_path_analysis:
    sample_nodes = nodes_list
    print(f"Analyzing paths for all {len(sample_nodes)} nodes")
else:
    # Sample nodes for analysis
    np.random.seed(42)  # For reproducible results
    sample_size = min(max_nodes_for_path_analysis, len(nodes_list))
    sample_nodes = np.random.choice(nodes_list, size=sample_size, replace=False)
    print(f"Analyzing paths for sample of {len(sample_nodes)} nodes (out of {len(nodes_list)} total)")

# Get the largest connected component for path analysis
if G.is_directed():
    if nx.is_strongly_connected(G):
        largest_component = G
        print("Using full graph (strongly connected)")
    else:
        wcc = max(nx.weakly_connected_components(G), key=len)
        largest_component = G.subgraph(wcc)
        print(f"Using largest weakly connected component ({len(wcc)} nodes)")
else:
    if nx.is_connected(G):
        largest_component = G
        print("Using full graph (connected)")
    else:
        cc = max(nx.connected_components(G), key=len)
        largest_component = G.subgraph(cc)
        print(f"Using largest connected component ({len(cc)} nodes)")

# Calculate shortest path lengths within the largest component
component_nodes = list(largest_component.nodes())
sample_component_nodes = [n for n in sample_nodes if n in component_nodes]

if len(sample_component_nodes) > 1:
    print(f"\n📏 SHORTEST PATH ANALYSIS:")
    print(f"Analyzing {len(sample_component_nodes)} nodes in largest component")
    
    # Calculate shortest path lengths
    path_lengths = []
    reachable_pairs = 0
    total_pairs = 0
    
    # Sample pairs to avoid excessive computation
    max_pairs = 1000
    pairs_to_check = min(max_pairs, len(sample_component_nodes) * (len(sample_component_nodes) - 1))
    
    print(f"Checking {pairs_to_check} node pairs for shortest paths...")
    
    for i, source in enumerate(sample_component_nodes[:int(np.sqrt(pairs_to_check))]):
        for target in sample_component_nodes[i+1:int(np.sqrt(pairs_to_check))]:
            total_pairs += 1
            try:
                if G.is_directed():
                    length = nx.shortest_path_length(largest_component, source, target)
                else:
                    length = nx.shortest_path_length(largest_component, source, target)
                path_lengths.append(length)
                reachable_pairs += 1
            except nx.NetworkXNoPath:
                pass  # Nodes not connected
    
    if path_lengths:
        print(f"\n📊 PATH LENGTH STATISTICS:")
        print(f"Total pairs checked: {total_pairs}")
        print(f"Reachable pairs: {reachable_pairs} ({reachable_pairs/total_pairs*100:.1f}%)")
        print(f"Average shortest path length: {np.mean(path_lengths):.2f}")
        print(f"Median shortest path length: {np.median(path_lengths):.2f}")
        print(f"Min path length: {min(path_lengths)}")
        print(f"Max path length: {max(path_lengths)}")
        print(f"Standard deviation: {np.std(path_lengths):.2f}")
        
        # Path length distribution
        path_length_counts = Counter(path_lengths)
        print(f"\nPath Length Distribution:")
        for length in sorted(path_length_counts.keys()):
            count = path_length_counts[length]
            percentage = (count / len(path_lengths)) * 100
            print(f"  Length {length}: {count} paths ({percentage:.1f}%)")
    
    # Diameter and radius (for smaller components)
    if len(component_nodes) <= 100:
        try:
            if G.is_directed():
                if nx.is_strongly_connected(largest_component):
                    diameter = nx.diameter(largest_component)
                    radius = nx.radius(largest_component)
                    print(f"\n🎯 GRAPH DIAMETER & RADIUS:")
                    print(f"Diameter: {diameter}")
                    print(f"Radius: {radius}")
                else:
                    print(f"\n⚠️ Cannot calculate diameter/radius: not strongly connected")
            else:
                diameter = nx.diameter(largest_component)
                radius = nx.radius(largest_component)
                print(f"\n🎯 GRAPH DIAMETER & RADIUS:")
                print(f"Diameter: {diameter}")
                print(f"Radius: {radius}")
        except nx.NetworkXError as e:
            print(f"\n⚠️ Cannot calculate diameter/radius: {e}")
    else:
        print(f"\n⚠️ Component too large ({len(component_nodes)} nodes) for diameter calculation")

else:
    print("⚠️ Insufficient nodes in largest component for path analysis")

# Average clustering coefficient
print(f"\n🕸️ CLUSTERING ANALYSIS:")
try:
    if G.is_directed():
        clustering = nx.average_clustering(G)
        print(f"Average Clustering Coefficient: {clustering:.4f}")
    else:
        clustering = nx.average_clustering(G)
        print(f"Average Clustering Coefficient: {clustering:.4f}")
        
    # Transitivity
    transitivity = nx.transitivity(G)
    print(f"Transitivity: {transitivity:.4f}")
    
except Exception as e:
    print(f"Could not calculate clustering: {e}")

print(f"\n✅ Path analysis completed!")

## 7. Centrality Measures

In [ ]:
# Calculate centrality measures to identify important nodes for connectivity
print("🎯 CENTRALITY MEASURES ANALYSIS")
print("=" * 50)

# For large graphs, we'll work with a subset or the largest component
max_nodes_for_centrality = 1000

if G.number_of_nodes() <= max_nodes_for_centrality:
    analysis_graph = G
    print(f"Calculating centrality for full graph ({G.number_of_nodes()} nodes)")
else:
    # Use largest connected component
    if G.is_directed():
        largest_cc = max(nx.weakly_connected_components(G), key=len)
    else:
        largest_cc = max(nx.connected_components(G), key=len)
    
    if len(largest_cc) <= max_nodes_for_centrality:
        analysis_graph = G.subgraph(largest_cc)
        print(f"Calculating centrality for largest component ({len(largest_cc)} nodes)")
    else:
        # Sample from largest component
        np.random.seed(42)
        sample_nodes = np.random.choice(list(largest_cc), size=max_nodes_for_centrality, replace=False)
        analysis_graph = G.subgraph(sample_nodes)
        print(f"Calculating centrality for sample of largest component ({max_nodes_for_centrality} nodes)")

print(f"Analysis graph: {analysis_graph.number_of_nodes()} nodes, {analysis_graph.number_of_edges()} edges")

# 1. Degree Centrality
print(f"\n📊 DEGREE CENTRALITY:")
degree_centrality = nx.degree_centrality(analysis_graph)
deg_cent_values = list(degree_centrality.values())

print(f"Average Degree Centrality: {np.mean(deg_cent_values):.4f}")
print(f"Max Degree Centrality: {max(deg_cent_values):.4f}")
print(f"Min Degree Centrality: {min(deg_cent_values):.4f}")
print(f"Standard Deviation: {np.std(deg_cent_values):.4f}")

# Top nodes by degree centrality
top_degree_nodes = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:5]
print(f"\nTop 5 nodes by Degree Centrality:")
for i, (node, centrality) in enumerate(top_degree_nodes, 1):
    print(f"  {i}. Node {node}: {centrality:.4f}")

# 2. Betweenness Centrality (for smaller graphs)
if analysis_graph.number_of_nodes() <= 500:
    print(f"\n🌉 BETWEENNESS CENTRALITY:")
    try:
        betweenness_centrality = nx.betweenness_centrality(analysis_graph, k=min(100, analysis_graph.number_of_nodes()))
        bet_cent_values = list(betweenness_centrality.values())
        
        print(f"Average Betweenness Centrality: {np.mean(bet_cent_values):.4f}")
        print(f"Max Betweenness Centrality: {max(bet_cent_values):.4f}")
        print(f"Min Betweenness Centrality: {min(bet_cent_values):.4f}")
        print(f"Standard Deviation: {np.std(bet_cent_values):.4f}")
        
        # Top nodes by betweenness centrality
        top_betweenness_nodes = sorted(betweenness_centrality.items(), key=lambda x: x[1], reverse=True)[:5]
        print(f"\nTop 5 nodes by Betweenness Centrality:")
        for i, (node, centrality) in enumerate(top_betweenness_nodes, 1):
            print(f"  {i}. Node {node}: {centrality:.4f}")
            
    except Exception as e:
        print(f"Could not calculate betweenness centrality: {e}")
else:
    print(f"\n⚠️ Graph too large for betweenness centrality calculation")

# 3. Closeness Centrality (for smaller connected components)
if analysis_graph.number_of_nodes() <= 500:
    print(f"\n🎯 CLOSENESS CENTRALITY:")
    try:
        # Check if graph is connected enough for closeness centrality
        if G.is_directed():
            if nx.is_strongly_connected(analysis_graph):
                closeness_centrality = nx.closeness_centrality(analysis_graph)
                calc_closeness = True
            else:
                print("Graph not strongly connected - calculating for largest SCC")
                scc = max(nx.strongly_connected_components(analysis_graph), key=len)
                if len(scc) >= 10:  # Only if component is reasonably large
                    closeness_centrality = nx.closeness_centrality(analysis_graph.subgraph(scc))
                    calc_closeness = True
                else:
                    calc_closeness = False
        else:
            if nx.is_connected(analysis_graph):
                closeness_centrality = nx.closeness_centrality(analysis_graph)
                calc_closeness = True
            else:
                print("Graph not connected - calculating for largest CC")
                cc = max(nx.connected_components(analysis_graph), key=len)
                if len(cc) >= 10:  # Only if component is reasonably large
                    closeness_centrality = nx.closeness_centrality(analysis_graph.subgraph(cc))
                    calc_closeness = True
                else:
                    calc_closeness = False
        
        if calc_closeness:
            close_cent_values = list(closeness_centrality.values())
            
            print(f"Average Closeness Centrality: {np.mean(close_cent_values):.4f}")
            print(f"Max Closeness Centrality: {max(close_cent_values):.4f}")
            print(f"Min Closeness Centrality: {min(close_cent_values):.4f}")
            print(f"Standard Deviation: {np.std(close_cent_values):.4f}")
            
            # Top nodes by closeness centrality
            top_closeness_nodes = sorted(closeness_centrality.items(), key=lambda x: x[1], reverse=True)[:5]
            print(f"\nTop 5 nodes by Closeness Centrality:")
            for i, (node, centrality) in enumerate(top_closeness_nodes, 1):
                print(f"  {i}. Node {node}: {centrality:.4f}")
        else:
            print("Cannot calculate closeness centrality - no suitable connected component")
            
    except Exception as e:
        print(f"Could not calculate closeness centrality: {e}")
else:
    print(f"\n⚠️ Graph too large for closeness centrality calculation")

# 4. Eigenvector Centrality (if possible)
print(f"\n🔢 EIGENVECTOR CENTRALITY:")
try:
    # Eigenvector centrality works best on undirected graphs or strongly connected directed graphs
    if G.is_directed():
        # Convert to undirected for eigenvector centrality
        undirected_graph = analysis_graph.to_undirected()
        eigenvector_centrality = nx.eigenvector_centrality(undirected_graph, max_iter=1000)
        print("(Calculated on undirected version of graph)")
    else:
        eigenvector_centrality = nx.eigenvector_centrality(analysis_graph, max_iter=1000)
    
    eig_cent_values = list(eigenvector_centrality.values())
    
    print(f"Average Eigenvector Centrality: {np.mean(eig_cent_values):.4f}")
    print(f"Max Eigenvector Centrality: {max(eig_cent_values):.4f}")
    print(f"Min Eigenvector Centrality: {min(eig_cent_values):.4f}")
    print(f"Standard Deviation: {np.std(eig_cent_values):.4f}")
    
    # Top nodes by eigenvector centrality
    top_eigenvector_nodes = sorted(eigenvector_centrality.items(), key=lambda x: x[1], reverse=True)[:5]
    print(f"\nTop 5 nodes by Eigenvector Centrality:")
    for i, (node, centrality) in enumerate(top_eigenvector_nodes, 1):
        print(f"  {i}. Node {node}: {centrality:.4f}")
        
except Exception as e:
    print(f"Could not calculate eigenvector centrality: {e}")

print(f"\n✅ Centrality analysis completed!")

## 8. Visualization of Connectivity

In [ ]:
# Create visualizations to show connectivity patterns
print("📊 CREATING CONNECTIVITY VISUALIZATIONS")
print("=" * 50)

# Set up the plotting
fig, axes = plt.subplots(2, 2, figsize=(20, 16))
fig.suptitle('Graph Connectivity Analysis Visualizations', fontsize=16, fontweight='bold')

# 1. Degree Distribution
ax1 = axes[0, 0]
degrees = [d for n, d in G.degree()]
degree_counts = Counter(degrees)

x_vals = list(degree_counts.keys())
y_vals = list(degree_counts.values())

ax1.bar(x_vals, y_vals, alpha=0.7, color='skyblue', edgecolor='navy')
ax1.set_xlabel('Degree')
ax1.set_ylabel('Number of Nodes')
ax1.set_title('Degree Distribution')
ax1.grid(True, alpha=0.3)

# Add statistics text
mean_degree = np.mean(degrees)
median_degree = np.median(degrees)
ax1.axvline(mean_degree, color='red', linestyle='--', alpha=0.7, label=f'Mean: {mean_degree:.1f}')
ax1.axvline(median_degree, color='orange', linestyle='--', alpha=0.7, label=f'Median: {median_degree:.1f}')
ax1.legend()

# 2. Connected Components Size Distribution
ax2 = axes[0, 1]
if G.is_directed():
    components = list(nx.weakly_connected_components(G))
    title = 'Weakly Connected Components Size Distribution'
else:
    components = list(nx.connected_components(G))
    title = 'Connected Components Size Distribution'

component_sizes = [len(comp) for comp in components]
size_counts = Counter(component_sizes)

if len(size_counts) > 0:
    x_vals = list(size_counts.keys())
    y_vals = list(size_counts.values())
    
    if len(x_vals) <= 20:  # Bar plot for few components
        ax2.bar(x_vals, y_vals, alpha=0.7, color='lightcoral', edgecolor='darkred')
        ax2.set_xlabel('Component Size')
        ax2.set_xticks(x_vals)
    else:  # Histogram for many components
        ax2.hist(component_sizes, bins=min(20, len(set(component_sizes))), alpha=0.7, color='lightcoral', edgecolor='darkred')
        ax2.set_xlabel('Component Size')
    
    ax2.set_ylabel('Number of Components')
    ax2.set_title(title)
    ax2.grid(True, alpha=0.3)
    
    # Add largest component info
    largest_size = max(component_sizes)
    ax2.axvline(largest_size, color='red', linestyle='--', alpha=0.7, 
                label=f'Largest: {largest_size} nodes')
    ax2.legend()
else:
    ax2.text(0.5, 0.5, 'No connected components\nto display', 
             horizontalalignment='center', verticalalignment='center', 
             transform=ax2.transAxes, fontsize=12)
    ax2.set_title(title)

# 3. Path Length Distribution (if calculated)
ax3 = axes[1, 0]
try:
    # Sample paths for visualization
    sample_size = min(200, G.number_of_nodes())
    sample_nodes = np.random.choice(list(G.nodes()), size=sample_size, replace=False)
    
    # Get largest component nodes for path calculation
    if G.is_directed():
        largest_comp = max(nx.weakly_connected_components(G), key=len)
    else:
        largest_comp = max(nx.connected_components(G), key=len)
    
    sample_comp_nodes = [n for n in sample_nodes if n in largest_comp]
    
    if len(sample_comp_nodes) > 10:
        subgraph = G.subgraph(largest_comp)
        path_lengths = []
        
        # Calculate sample of path lengths
        for i in range(min(100, len(sample_comp_nodes))):
            for j in range(i+1, min(i+10, len(sample_comp_nodes))):
                try:
                    length = nx.shortest_path_length(subgraph, sample_comp_nodes[i], sample_comp_nodes[j])
                    path_lengths.append(length)
                except nx.NetworkXNoPath:
                    pass
        
        if path_lengths:
            ax3.hist(path_lengths, bins=range(1, max(path_lengths)+2), alpha=0.7, 
                    color='lightgreen', edgecolor='darkgreen')
            ax3.set_xlabel('Path Length')
            ax3.set_ylabel('Frequency')
            ax3.set_title('Shortest Path Length Distribution\n(Sample)')
            ax3.grid(True, alpha=0.3)
            
            # Add mean
            mean_path = np.mean(path_lengths)
            ax3.axvline(mean_path, color='red', linestyle='--', alpha=0.7, 
                       label=f'Mean: {mean_path:.1f}')
            ax3.legend()
        else:
            ax3.text(0.5, 0.5, 'No paths found\nin sample', 
                    horizontalalignment='center', verticalalignment='center', 
                    transform=ax3.transAxes, fontsize=12)
            ax3.set_title('Shortest Path Length Distribution')
    else:
        ax3.text(0.5, 0.5, 'Insufficient nodes\nfor path analysis', 
                horizontalalignment='center', verticalalignment='center', 
                transform=ax3.transAxes, fontsize=12)
        ax3.set_title('Shortest Path Length Distribution')
        
except Exception as e:
    ax3.text(0.5, 0.5, f'Path analysis\nnot available:\n{str(e)[:30]}...', 
            horizontalalignment='center', verticalalignment='center', 
            transform=ax3.transAxes, fontsize=10)
    ax3.set_title('Shortest Path Length Distribution')

# 4. Network Summary Statistics
ax4 = axes[1, 1]
ax4.axis('off')  # Turn off axis for text display

# Prepare summary statistics
stats_text = []
stats_text.append("📊 NETWORK CONNECTIVITY SUMMARY")
stats_text.append("=" * 40)
stats_text.append(f"Nodes: {G.number_of_nodes():,}")
stats_text.append(f"Edges: {G.number_of_edges():,}")
stats_text.append(f"Graph Type: {'Directed' if G.is_directed() else 'Undirected'}")
stats_text.append(f"Density: {nx.density(G):.6f}")

# Connectivity status
if G.is_directed():
    stats_text.append(f"Strongly Connected: {nx.is_strongly_connected(G)}")
    stats_text.append(f"Weakly Connected: {nx.is_weakly_connected(G)}")
    if not nx.is_weakly_connected(G):
        stats_text.append(f"WCC Count: {nx.number_weakly_connected_components(G)}")
else:
    stats_text.append(f"Connected: {nx.is_connected(G)}")
    if not nx.is_connected(G):
        stats_text.append(f"CC Count: {nx.number_connected_components(G)}")

# Degree statistics
degrees = [d for n, d in G.degree()]
stats_text.append("")
stats_text.append("Degree Statistics:")
stats_text.append(f"  Mean: {np.mean(degrees):.2f}")
stats_text.append(f"  Median: {np.median(degrees):.2f}")
stats_text.append(f"  Min: {min(degrees)}")
stats_text.append(f"  Max: {max(degrees)}")

# Clustering (if available)
try:
    clustering = nx.average_clustering(G)
    stats_text.append(f"  Clustering: {clustering:.4f}")
except:
    pass

# Display text
ax4.text(0.05, 0.95, '\n'.join(stats_text), 
         transform=ax4.transAxes, fontsize=11, 
         verticalalignment='top', horizontalalignment='left',
         family='monospace',
         bbox=dict(boxstyle="round,pad=0.5", facecolor="lightgray", alpha=0.8))

plt.tight_layout()
plt.show()

print("✅ Connectivity visualizations created!")

# Create a simple network layout visualization for smaller graphs
if G.number_of_nodes() <= 100:
    print(f"\n🕸️ Creating network layout visualization...")
    
    plt.figure(figsize=(12, 8))
    
    # Choose layout based on graph size
    if G.number_of_nodes() <= 20:
        pos = nx.spring_layout(G, k=2, iterations=50)
    else:
        pos = nx.spring_layout(G, k=1, iterations=30)
    
    # Color nodes by connected component
    if G.is_directed():
        components = list(nx.weakly_connected_components(G))
    else:
        components = list(nx.connected_components(G))
    
    # Create color map
    colors = plt.cm.Set3(np.linspace(0, 1, len(components)))
    node_colors = []
    
    for node in G.nodes():
        for i, component in enumerate(components):
            if node in component:
                node_colors.append(colors[i])
                break
    
    # Draw the graph
    nx.draw_networkx_nodes(G, pos, node_color=node_colors, node_size=100, alpha=0.8)
    nx.draw_networkx_edges(G, pos, alpha=0.5, width=0.5)
    
    if G.number_of_nodes() <= 30:
        nx.draw_networkx_labels(G, pos, font_size=8, font_weight='bold')
    
    plt.title(f'Network Layout - {len(components)} Connected Component(s)', fontsize=14, fontweight='bold')
    plt.axis('off')
    plt.tight_layout()
    plt.show()
    
    print("✅ Network layout visualization created!")
    
elif G.number_of_nodes() <= 500:
    print(f"\n🕸️ Creating simplified network visualization for {G.number_of_nodes()} nodes...")
    
    # Sample nodes for visualization
    sample_size = min(100, G.number_of_nodes())
    sample_nodes = np.random.choice(list(G.nodes()), size=sample_size, replace=False)
    subgraph = G.subgraph(sample_nodes)
    
    plt.figure(figsize=(10, 8))
    pos = nx.spring_layout(subgraph, k=1, iterations=30)
    
    nx.draw_networkx_nodes(subgraph, pos, node_color='lightblue', node_size=50, alpha=0.8)
    nx.draw_networkx_edges(subgraph, pos, alpha=0.3, width=0.5)
    
    plt.title(f'Network Sample ({sample_size} nodes)', fontsize=14, fontweight='bold')
    plt.axis('off')
    plt.tight_layout()
    plt.show()
    
    print("✅ Sample network visualization created!")
    
else:
    print(f"⚠️ Graph too large ({G.number_of_nodes()} nodes) for network layout visualization")

print(f"\n🎉 CONNECTIVITY ANALYSIS COMPLETE!")
print("=" * 50)

## 9. Layer 0 Connectivity Analysis

This section analyzes the paths and connectivity between all nodes in Layer 0 (ground level). This is particularly important for understanding:
- Horizontal connectivity at the lowest altitude
- Path availability for low-altitude UAS operations
- Network efficiency at ground level

In [ ]:
# Comprehensive analysis of paths between all Layer 0 nodes
print("🏢 LAYER 0 CONNECTIVITY ANALYSIS")
print("=" * 50)

# First, identify Layer 0 nodes
layer_0_nodes = []
layer_attr = None

# Try to find layer attribute
if G.nodes():
    sample_node = list(G.nodes())[0]
    node_attrs = G.nodes[sample_node]
    
    # Check for common layer attribute names
    for attr in ['layer', 'altitude', 'z', 'level', 'floor']:
        if attr in node_attrs:
            layer_attr = attr
            break
    
    if layer_attr:
        print(f"✅ Found layer attribute: '{layer_attr}'")
        
        # Extract all Layer 0 nodes
        for node in G.nodes():
            layer = G.nodes[node].get(layer_attr)
            if layer == 0:
                layer_0_nodes.append(node)
        
        print(f"Found {len(layer_0_nodes)} nodes in Layer 0")
    else:
        print("❌ Could not identify layer attribute")
        print("Available node attributes:", list(node_attrs.keys()))

if len(layer_0_nodes) == 0:
    print("\n⚠️ No Layer 0 nodes found. Unable to perform analysis.")
else:
    # Create subgraph of Layer 0 nodes
    layer_0_subgraph = G.subgraph(layer_0_nodes)
    
    print(f"\n📊 LAYER 0 SUBGRAPH STATISTICS:")
    print(f"Nodes in Layer 0: {layer_0_subgraph.number_of_nodes()}")
    print(f"Edges in Layer 0: {layer_0_subgraph.number_of_edges()}")
    print(f"Graph Type: {'Directed' if layer_0_subgraph.is_directed() else 'Undirected'}")
    
    # Connectivity check
    if layer_0_subgraph.is_directed():
        is_strongly_connected = nx.is_strongly_connected(layer_0_subgraph)
        is_weakly_connected = nx.is_weakly_connected(layer_0_subgraph)
        print(f"Strongly Connected: {is_strongly_connected}")
        print(f"Weakly Connected: {is_weakly_connected}")
        
        if not is_weakly_connected:
            num_wcc = nx.number_weakly_connected_components(layer_0_subgraph)
            print(f"Number of Weakly Connected Components: {num_wcc}")
    else:
        is_connected = nx.is_connected(layer_0_subgraph)
        print(f"Connected: {is_connected}")
        
        if not is_connected:
            num_cc = nx.number_connected_components(layer_0_subgraph)
            print(f"Number of Connected Components: {num_cc}")
    
    # Degree distribution at Layer 0
    layer_0_degrees = [d for n, d in layer_0_subgraph.degree()]
    if layer_0_degrees:
        print(f"\n📈 LAYER 0 DEGREE STATISTICS:")
        print(f"Average Degree: {np.mean(layer_0_degrees):.2f}")
        print(f"Median Degree: {np.median(layer_0_degrees):.2f}")
        print(f"Min Degree: {min(layer_0_degrees)}")
        print(f"Max Degree: {max(layer_0_degrees)}")
        
        degree_counts = Counter(layer_0_degrees)
        print(f"\nDegree Distribution:")
        for degree in sorted(degree_counts.keys()):
            count = degree_counts[degree]
            percentage = (count / len(layer_0_degrees)) * 100
            print(f"  Degree {degree}: {count} nodes ({percentage:.1f}%)")
    
    # Path analysis between all Layer 0 nodes
    print(f"\n🛤️ PATH ANALYSIS BETWEEN ALL LAYER 0 NODES:")
    
    layer_0_node_list = list(layer_0_nodes)
    total_pairs = len(layer_0_node_list) * (len(layer_0_node_list) - 1)
    if not layer_0_subgraph.is_directed():
        total_pairs = total_pairs // 2
    
    print(f"Total node pairs to analyze: {total_pairs:,}")
    
    # For very large Layer 0, we might need to sample
    max_pairs_to_check = 10000
    
    if total_pairs <= max_pairs_to_check:
        print(f"Analyzing all {total_pairs:,} pairs...")
        analyze_all = True
    else:
        print(f"Layer 0 is large - analyzing sample of {max_pairs_to_check:,} pairs...")
        analyze_all = False
    
    path_lengths = []
    reachable_pairs = 0
    unreachable_pairs = 0
    pairs_checked = 0
    
    # Get the largest connected component for path analysis
    if layer_0_subgraph.is_directed():
        if nx.is_weakly_connected(layer_0_subgraph):
            path_graph = layer_0_subgraph
        else:
            wcc = max(nx.weakly_connected_components(layer_0_subgraph), key=len)
            path_graph = layer_0_subgraph.subgraph(wcc)
            print(f"Using largest weakly connected component: {len(wcc)} nodes")
    else:
        if nx.is_connected(layer_0_subgraph):
            path_graph = layer_0_subgraph
        else:
            cc = max(nx.connected_components(layer_0_subgraph), key=len)
            path_graph = layer_0_subgraph.subgraph(cc)
            print(f"Using largest connected component: {len(cc)} nodes")
    
    path_graph_nodes = list(path_graph.nodes())
    
    # Calculate paths
    if analyze_all:
        # Check all pairs
        for i, source in enumerate(path_graph_nodes):
            for j, target in enumerate(path_graph_nodes):
                if source == target:
                    continue
                
                if not layer_0_subgraph.is_directed() and j <= i:
                    continue  # Skip duplicate pairs in undirected graph
                
                pairs_checked += 1
                
                try:
                    length = nx.shortest_path_length(path_graph, source, target)
                    path_lengths.append(length)
                    reachable_pairs += 1
                except nx.NetworkXNoPath:
                    unreachable_pairs += 1
                
                # Progress indicator for large graphs
                if pairs_checked % 1000 == 0:
                    print(f"  Processed {pairs_checked:,} / {total_pairs:,} pairs...", end='\r')
    else:
        # Sample pairs
        np.random.seed(42)
        sample_size = int(np.sqrt(max_pairs_to_check))
        sample_nodes = np.random.choice(path_graph_nodes, size=min(sample_size, len(path_graph_nodes)), replace=False)
        
        for i, source in enumerate(sample_nodes):
            for j, target in enumerate(sample_nodes):
                if source == target:
                    continue
                
                if not layer_0_subgraph.is_directed() and j <= i:
                    continue
                
                pairs_checked += 1
                
                try:
                    length = nx.shortest_path_length(path_graph, source, target)
                    path_lengths.append(length)
                    reachable_pairs += 1
                except nx.NetworkXNoPath:
                    unreachable_pairs += 1
    
    print(f"\n")  # Clear progress line
    
    # Display results
    print(f"📊 PATH ANALYSIS RESULTS:")
    print(f"Pairs checked: {pairs_checked:,}")
    
    if pairs_checked > 0:
        print(f"Reachable pairs: {reachable_pairs:,} ({reachable_pairs/pairs_checked*100:.1f}%)")
        print(f"Unreachable pairs: {unreachable_pairs:,} ({unreachable_pairs/pairs_checked*100:.1f}%)")
    else:
        print(f"Reachable pairs: {reachable_pairs:,} (No pairs to analyze)")
        print(f"Unreachable pairs: {unreachable_pairs:,} (No pairs to analyze)")
    
    if path_lengths:
        print(f"\n📏 PATH LENGTH STATISTICS:")
        print(f"Average shortest path: {np.mean(path_lengths):.2f}")
        print(f"Median shortest path: {np.median(path_lengths):.2f}")
        print(f"Min path length: {min(path_lengths)}")
        print(f"Max path length: {max(path_lengths)}")
        print(f"Standard deviation: {np.std(path_lengths):.2f}")
        
        # Path length distribution
        path_length_counts = Counter(path_lengths)
        print(f"\n📊 PATH LENGTH DISTRIBUTION:")
        for length in sorted(path_length_counts.keys()):
            count = path_length_counts[length]
            percentage = (count / len(path_lengths)) * 100
            print(f"  Length {length}: {count} paths ({percentage:.1f}%)")
        
        # Calculate diameter and radius if possible
        if len(path_graph_nodes) <= 500 and reachable_pairs > 0:
            try:
                if layer_0_subgraph.is_directed():
                    if nx.is_strongly_connected(path_graph):
                        diameter = nx.diameter(path_graph)
                        radius = nx.radius(path_graph)
                        print(f"\n🎯 LAYER 0 GRAPH METRICS:")
                        print(f"Diameter (longest shortest path): {diameter}")
                        print(f"Radius: {radius}")
                        
                        # Find center and periphery nodes
                        center = nx.center(path_graph)
                        periphery = nx.periphery(path_graph)
                        print(f"Center nodes: {len(center)}")
                        print(f"Periphery nodes: {len(periphery)}")
                else:
                    if nx.is_connected(path_graph):
                        diameter = nx.diameter(path_graph)
                        radius = nx.radius(path_graph)
                        print(f"\n🎯 LAYER 0 GRAPH METRICS:")
                        print(f"Diameter (longest shortest path): {diameter}")
                        print(f"Radius: {radius}")
                        
                        # Find center and periphery nodes
                        center = nx.center(path_graph)
                        periphery = nx.periphery(path_graph)
                        print(f"Center nodes: {len(center)}")
                        print(f"Periphery nodes: {len(periphery)}")
            except Exception as e:
                print(f"\n⚠️ Could not calculate diameter/radius: {e}")
        
        # Visualize path length distribution
        print(f"\n📊 Creating Layer 0 path length visualization...")
        
        fig, axes = plt.subplots(1, 2, figsize=(16, 6))
        fig.suptitle('Layer 0 Connectivity Analysis', fontsize=14, fontweight='bold')
        
        # Path length histogram
        ax1 = axes[0]
        ax1.hist(path_lengths, bins=range(1, max(path_lengths)+2), alpha=0.7, 
                color='steelblue', edgecolor='navy')
        ax1.set_xlabel('Path Length (hops)')
        ax1.set_ylabel('Number of Paths')
        ax1.set_title('Layer 0: Shortest Path Length Distribution')
        ax1.grid(True, alpha=0.3)
        
        mean_path = np.mean(path_lengths)
        ax1.axvline(mean_path, color='red', linestyle='--', alpha=0.7, 
                   label=f'Mean: {mean_path:.2f}')
        ax1.legend()
        
        # Summary statistics
        ax2 = axes[1]
        ax2.axis('off')
        
        summary_text = []
        summary_text.append("📊 LAYER 0 SUMMARY")
        summary_text.append("=" * 35)
        summary_text.append(f"Total Nodes: {len(layer_0_nodes):,}")
        summary_text.append(f"Total Edges: {layer_0_subgraph.number_of_edges():,}")
        summary_text.append(f"Density: {nx.density(layer_0_subgraph):.6f}")
        summary_text.append("")
        summary_text.append("Connectivity:")
        if layer_0_subgraph.is_directed():
            summary_text.append(f"  Strongly: {nx.is_strongly_connected(layer_0_subgraph)}")
            summary_text.append(f"  Weakly: {nx.is_weakly_connected(layer_0_subgraph)}")
        else:
            summary_text.append(f"  Connected: {nx.is_connected(layer_0_subgraph)}")
        summary_text.append("")
        summary_text.append("Path Statistics:")
        summary_text.append(f"  Pairs checked: {pairs_checked:,}")
        summary_text.append(f"  Reachable: {reachable_pairs:,}")
        summary_text.append(f"  Unreachable: {unreachable_pairs:,}")
        if pairs_checked > 0:
            summary_text.append(f"  Reachability: {reachable_pairs/pairs_checked*100:.1f}%")
        else:
            summary_text.append(f"  Reachability: No pairs to analyze")
        summary_text.append("")
        summary_text.append(f"  Avg path length: {np.mean(path_lengths):.2f}")
        summary_text.append(f"  Max path length: {max(path_lengths)}")
        summary_text.append(f"  Min path length: {min(path_lengths)}")
        
        ax2.text(0.05, 0.95, '\n'.join(summary_text), 
                transform=ax2.transAxes, fontsize=11,
                verticalalignment='top', horizontalalignment='left',
                family='monospace',
                bbox=dict(boxstyle="round,pad=0.5", facecolor="lightblue", alpha=0.8))
        
        plt.tight_layout()
        plt.show()
        
        print("✅ Layer 0 visualization created!")
    
    else:
        print(f"\n⚠️ No paths found between Layer 0 nodes")
    
    # Network efficiency at Layer 0
    if reachable_pairs > 0 and path_lengths:
        avg_path_length = np.mean(path_lengths)
        max_possible_efficiency = 1.0  # Direct connection
        actual_efficiency = 1.0 / avg_path_length
        
        print(f"\n⚡ LAYER 0 NETWORK EFFICIENCY:")
        print(f"Average path efficiency: {actual_efficiency:.4f}")
        print(f"Efficiency ratio: {actual_efficiency/max_possible_efficiency:.1%}")
        
        # Calculate network robustness indicators
        if len(layer_0_degrees) > 0:
            degree_variance = np.var(layer_0_degrees)
            degree_std = np.std(layer_0_degrees)
            
            print(f"\n🛡️ LAYER 0 ROBUSTNESS INDICATORS:")
            print(f"Degree variance: {degree_variance:.2f}")
            print(f"Degree std dev: {degree_std:.2f}")
            
            if degree_std < 1.0:
                print(f"  → Highly regular structure (uniform connectivity)")
            elif degree_std < 2.0:
                print(f"  → Moderately regular structure")
            else:
                print(f"  → Irregular structure (varying connectivity)")

print(f"\n✅ Layer 0 connectivity analysis completed!")

## 9.1 Layer 0 Multi-Layer Path Analysis

This section analyzes whether Layer 0 nodes can reach other Layer 0 nodes by using paths that traverse through higher layers. This is crucial for understanding:
- Multi-layer connectivity starting and ending at ground level
- Whether UAS can reach any Layer 0 destination from any Layer 0 origin using vertical movement
- Path diversity and redundancy through the 3D airspace

In [ ]:
# Analysis of Layer 0 to Layer 0 paths using the full 3D graph (including vertical paths)
print("🌐 LAYER 0 MULTI-LAYER PATH ANALYSIS")
print("=" * 50)

# Use the variables from the previous Layer 0 analysis
if len(layer_0_nodes) == 0:
    print("⚠️ No Layer 0 nodes found. Cannot perform multi-layer path analysis.")
else:
    print(f"Analyzing multi-layer paths between {len(layer_0_nodes)} Layer 0 nodes...")
    print(f"Using full 3D graph with {G.number_of_nodes():,} total nodes and {G.number_of_edges():,} edges")
    
    # Sample Layer 0 nodes if there are too many
    max_layer_0_nodes = 100  # Limit for computational efficiency
    
    if len(layer_0_nodes) <= max_layer_0_nodes:
        sample_layer_0_nodes = layer_0_nodes
        print(f"Analyzing all {len(sample_layer_0_nodes)} Layer 0 nodes")
    else:
        np.random.seed(42)
        sample_layer_0_nodes = np.random.choice(layer_0_nodes, size=max_layer_0_nodes, replace=False).tolist()
        print(f"Analyzing sample of {len(sample_layer_0_nodes)} Layer 0 nodes (out of {len(layer_0_nodes)} total)")
    
    # Calculate paths between Layer 0 nodes using the full 3D graph
    total_layer_0_pairs = len(sample_layer_0_nodes) * (len(sample_layer_0_nodes) - 1)
    if not G.is_directed():
        total_layer_0_pairs = total_layer_0_pairs // 2
    
    print(f"Total Layer 0 to Layer 0 pairs to analyze: {total_layer_0_pairs:,}")
    
    # Initialize counters
    multilayer_path_lengths = []
    multilayer_reachable_pairs = 0
    multilayer_unreachable_pairs = 0
    multilayer_pairs_checked = 0
    paths_using_other_layers = 0
    direct_layer_0_paths = 0
    
    # Track path details
    path_layer_usage = {}  # layer -> count of paths using that layer
    shortest_multilayer_path = None
    longest_multilayer_path = None
    
    print(f"\n🔍 ANALYZING MULTI-LAYER CONNECTIVITY:")
    print(f"Checking paths from Layer 0 to Layer 0 through the entire 3D graph...")
    
    # Calculate paths using the full graph
    for i, source in enumerate(sample_layer_0_nodes):
        for j, target in enumerate(sample_layer_0_nodes):
            if source == target:
                continue
            
            if not G.is_directed() and j <= i:
                continue  # Skip duplicate pairs in undirected graph
            
            multilayer_pairs_checked += 1
            
            try:
                # Find shortest path through the entire 3D graph
                path = nx.shortest_path(G, source, target)
                path_length = len(path) - 1  # Number of edges
                multilayer_path_lengths.append(path_length)
                multilayer_reachable_pairs += 1
                
                # Analyze which layers this path uses
                path_layers = set()
                layers_in_path = []
                uses_non_layer_0 = False
                
                for node in path:
                    node_layer = node_layers.get(node, 'unknown')
                    if node_layer != 'unknown':
                        path_layers.add(node_layer)
                        layers_in_path.append(node_layer)
                        if node_layer != 0:
                            uses_non_layer_0 = True
                
                # Count layer usage
                for layer in path_layers:
                    if layer not in path_layer_usage:
                        path_layer_usage[layer] = 0
                    path_layer_usage[layer] += 1
                
                # Categorize path type
                if uses_non_layer_0:
                    paths_using_other_layers += 1
                else:
                    direct_layer_0_paths += 1
                
                # Track shortest and longest paths
                if shortest_multilayer_path is None or path_length < len(shortest_multilayer_path) - 1:
                    shortest_multilayer_path = path
                if longest_multilayer_path is None or path_length > len(longest_multilayer_path) - 1:
                    longest_multilayer_path = path
                
            except nx.NetworkXNoPath:
                multilayer_unreachable_pairs += 1
            
            # Progress indicator
            if multilayer_pairs_checked % 100 == 0:
                print(f"  Processed {multilayer_pairs_checked:,} / {total_layer_0_pairs:,} pairs...", end='\r')
    
    print(f"\n")  # Clear progress line
    
    # Display results
    print(f"📊 MULTI-LAYER PATH RESULTS:")
    print(f"Layer 0 pairs checked: {multilayer_pairs_checked:,}")
    
    if multilayer_pairs_checked > 0:
        print(f"Reachable via multi-layer paths: {multilayer_reachable_pairs:,} ({multilayer_reachable_pairs/multilayer_pairs_checked*100:.1f}%)")
        print(f"Unreachable pairs: {multilayer_unreachable_pairs:,} ({multilayer_unreachable_pairs/multilayer_pairs_checked*100:.1f}%)")
        
        if multilayer_reachable_pairs > 0:
            print(f"\n🛤️ PATH TYPE BREAKDOWN:")
            print(f"Direct Layer 0 paths: {direct_layer_0_paths:,} ({direct_layer_0_paths/multilayer_reachable_pairs*100:.1f}%)")
            print(f"Paths using other layers: {paths_using_other_layers:,} ({paths_using_other_layers/multilayer_reachable_pairs*100:.1f}%)")
            
            print(f"\n📏 MULTI-LAYER PATH STATISTICS:")
            print(f"Average path length: {np.mean(multilayer_path_lengths):.2f}")
            print(f"Median path length: {np.median(multilayer_path_lengths):.2f}")
            print(f"Min path length: {min(multilayer_path_lengths)}")
            print(f"Max path length: {max(multilayer_path_lengths)}")
            print(f"Standard deviation: {np.std(multilayer_path_lengths):.2f}")
            
            # Path length distribution
            multilayer_path_counts = Counter(multilayer_path_lengths)
            print(f"\n📊 MULTI-LAYER PATH LENGTH DISTRIBUTION:")
            for length in sorted(multilayer_path_counts.keys())[:10]:  # Show first 10
                count = multilayer_path_counts[length]
                percentage = (count / len(multilayer_path_lengths)) * 100
                print(f"  Length {length}: {count} paths ({percentage:.1f}%)")
            
            # Layer usage analysis
            if path_layer_usage:
                print(f"\n🏗️ LAYER USAGE IN PATHS:")
                print(f"Layers used in Layer 0→Layer 0 paths:")
                for layer in sorted(path_layer_usage.keys()):
                    count = path_layer_usage[layer]
                    percentage = (count / multilayer_reachable_pairs) * 100
                    print(f"  Layer {layer}: {count} paths ({percentage:.1f}%)")
            
            # Example paths
            if shortest_multilayer_path and longest_multilayer_path:
                print(f"\n🎯 EXAMPLE PATHS:")
                
                # Shortest path
                shortest_layers = [node_layers.get(node, 'unknown') for node in shortest_multilayer_path]
                print(f"Shortest path ({len(shortest_multilayer_path)-1} hops):")
                print(f"  Nodes: {' → '.join(shortest_multilayer_path[:5])}{'...' if len(shortest_multilayer_path) > 5 else ''}")
                print(f"  Layers: {' → '.join(map(str, shortest_layers[:5]))}{'...' if len(shortest_layers) > 5 else ''}")
                
                # Longest path (if different and not too long)
                if len(longest_multilayer_path) != len(shortest_multilayer_path) and len(longest_multilayer_path) <= 20:
                    longest_layers = [node_layers.get(node, 'unknown') for node in longest_multilayer_path]
                    print(f"\nLongest path ({len(longest_multilayer_path)-1} hops):")
                    print(f"  Nodes: {' → '.join(longest_multilayer_path[:5])}{'...' if len(longest_multilayer_path) > 5 else ''}")
                    print(f"  Layers: {' → '.join(map(str, longest_layers[:5]))}{'...' if len(longest_layers) > 5 else ''}")
            
            # Comparison with Layer 0 only paths
            if 'path_lengths' in locals() and len(path_lengths) > 0:
                layer_0_only_avg = np.mean(path_lengths)
                multilayer_avg = np.mean(multilayer_path_lengths)
                
                print(f"\n🔄 COMPARISON: Layer 0 Only vs Multi-Layer Paths:")
                print(f"Layer 0 only average path length: {layer_0_only_avg:.2f}")
                print(f"Multi-layer average path length: {multilayer_avg:.2f}")
                
                if multilayer_avg < layer_0_only_avg:
                    improvement = ((layer_0_only_avg - multilayer_avg) / layer_0_only_avg) * 100
                    print(f"✅ Multi-layer paths are {improvement:.1f}% shorter on average")
                elif multilayer_avg > layer_0_only_avg:
                    increase = ((multilayer_avg - layer_0_only_avg) / layer_0_only_avg) * 100
                    print(f"⚠️ Multi-layer paths are {increase:.1f}% longer on average")
                else:
                    print(f"➡️ Multi-layer and Layer 0 only paths have similar lengths")
                
                # Reachability comparison
                layer_0_reachability = reachable_pairs / pairs_checked * 100 if pairs_checked > 0 else 0
                multilayer_reachability = multilayer_reachable_pairs / multilayer_pairs_checked * 100
                
                print(f"\nReachability comparison:")
                print(f"Layer 0 only: {layer_0_reachability:.1f}% reachable")
                print(f"Multi-layer: {multilayer_reachability:.1f}% reachable")
                
                if multilayer_reachability > layer_0_reachability:
                    improvement = multilayer_reachability - layer_0_reachability
                    print(f"✅ Multi-layer connectivity improves reachability by {improvement:.1f} percentage points")
                else:
                    print(f"➡️ Similar reachability between approaches")
    
    else:
        print(f"No Layer 0 pairs to analyze")
    
    # Create visualization
    if multilayer_path_lengths:
        print(f"\n📊 Creating multi-layer path visualization...")
        
        fig, axes = plt.subplots(1, 3, figsize=(18, 6))
        fig.suptitle('Layer 0 Multi-Layer Path Analysis', fontsize=14, fontweight='bold')
        
        # Path length distribution
        ax1 = axes[0]
        ax1.hist(multilayer_path_lengths, bins=range(1, min(max(multilayer_path_lengths)+2, 21)), 
                alpha=0.7, color='green', edgecolor='darkgreen')
        ax1.set_xlabel('Path Length (hops)')
        ax1.set_ylabel('Number of Paths')
        ax1.set_title('Multi-Layer Path Length Distribution')
        ax1.grid(True, alpha=0.3)
        
        mean_multilayer = np.mean(multilayer_path_lengths)
        ax1.axvline(mean_multilayer, color='red', linestyle='--', alpha=0.7, 
                   label=f'Mean: {mean_multilayer:.2f}')
        ax1.legend()
        
        # Path type comparison
        ax2 = axes[1]
        path_types = ['Direct\nLayer 0', 'Using Other\nLayers']
        path_counts = [direct_layer_0_paths, paths_using_other_layers]
        colors = ['lightblue', 'lightcoral']
        
        bars = ax2.bar(path_types, path_counts, color=colors, alpha=0.7, edgecolor='black')
        ax2.set_ylabel('Number of Paths')
        ax2.set_title('Path Type Distribution')
        ax2.grid(True, alpha=0.3)
        
        # Add value labels on bars
        for bar, count in zip(bars, path_counts):
            height = bar.get_height()
            ax2.text(bar.get_x() + bar.get_width()/2., height + max(path_counts)*0.01,
                    f'{count}', ha='center', va='bottom')
        
        # Layer usage
        ax3 = axes[2]
        if path_layer_usage:
            layers = sorted(path_layer_usage.keys())
            usage_counts = [path_layer_usage[layer] for layer in layers]
            
            ax3.bar([f'Layer {layer}' for layer in layers], usage_counts, 
                   alpha=0.7, color='orange', edgecolor='darkorange')
            ax3.set_xlabel('Layer')
            ax3.set_ylabel('Number of Paths Using Layer')
            ax3.set_title('Layer Usage in Paths')
            ax3.grid(True, alpha=0.3)
            plt.setp(ax3.get_xticklabels(), rotation=45)
        else:
            ax3.text(0.5, 0.5, 'No layer usage\ndata available', 
                    horizontalalignment='center', verticalalignment='center',
                    transform=ax3.transAxes, fontsize=12)
            ax3.set_title('Layer Usage in Paths')
        
        plt.tight_layout()
        plt.show()
        
        print("✅ Multi-layer path visualization created!")

print(f"\n✅ Layer 0 multi-layer path analysis completed!")

## Summary and Conclusions

This notebook provides a comprehensive analysis of the 3D lattice graph connectivity for UAS path planning. The analysis includes:

### Key Insights:
- **Graph Structure**: Detailed examination of the graph's basic properties including node/edge counts, degree distribution, and graph type
- **Connectivity Status**: Assessment of whether the graph is connected (for undirected) or strongly/weakly connected (for directed)
- **Component Analysis**: Identification and characterization of connected components and their sizes
- **Path Efficiency**: Analysis of shortest paths, diameter, and average path lengths within the network
- **Critical Nodes**: Identification of important nodes through various centrality measures
- **Network Visualization**: Visual representations of connectivity patterns and graph structure

### Applications for UAS Path Planning:
- **Route Planning**: Connected components indicate which areas of the airspace are reachable from each other
- **Network Resilience**: Centrality measures help identify critical nodes whose removal would most impact connectivity
- **Path Optimization**: Shortest path analysis provides insights into the most efficient routes through the 3D airspace
- **Airspace Utilization**: Density and clustering metrics show how well the lattice structure covers the available airspace

### Next Steps:
- Consider adding temporal or weather-based constraints to the connectivity analysis
- Analyze connectivity at different altitude layers separately
- Implement dynamic path planning algorithms using the connectivity insights
- Evaluate the impact of obstacles or no-fly zones on network connectivity